## Define the Posterior

Inference of the DDM parameters $\theta$ is performed by passing the trained MNLE to a posterior function. DDM-STRIDE extends the MCMC Posterior provided by the [sbi package](https://pypi.org/project/sbi/) by allowing inference over parameters given not only the observations $x$ but also experimental conditions $\pi$. [Markov chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) (MCMC) methods allow to draw correlated samples $\Theta$ with a probability that depends on the synthetic likelihood $q(x | \theta, \pi)$ and the prior $p(\theta)$.
Since likelihood and prior are proportional to the posterior, i.e. $p(\theta | x, \pi) \propto q(x | \theta, \pi) \cdot p(\theta)$, the samples over time approximate the posterior distribution.

Open your file in *config/task*. `posterior_params` to control how the posterior samples are drawn. You can find the parameters and their description in the [sbi package](https://github.com/mackelab/sbi/blob/7799de5e4bc676ef4a9db304b225503126735f2c/sbi/inference/posteriors/mcmc_posterior.py#L57). 

Available MCMC methods include Slice Sampling, Hamiltonian Monte Carlo and the No-U-Turn Sampler. `num_chains` determines how many independent MCMC chains will be run. Increasing the number of chains might yield more precise results especially in case of multimodal distributions. At the start of each chain, a number of `warmup_steps` is performed in order to move towards regions with higher probability before starting to keep the samples. Strongly autocorrelated MCMC chains might lead to many similar samples, hence causing a worse coverage of the posterior. The `thin` parameter allows to reduce the autocorrelation by only accepting every nth sample of a chain. When using `slice_np_vectorized`, the thinning factor is applied to the warmup steps, i.e. with 20 warmup steps and a thinning factor of 5 each chain will perform 100 warmup steps.  
As an `init_strategy` for each chain, initial locations can either be drawn from the proposal distribution or selected via Sequential-Importance-Resampling. SIR draws locations from `init_strategy_num_candidates` proportional to their probability. 

`num_workers` allows to parallelize sampling and speed up sampling. Since every worker takes up some working memory, you might want to keep an eye on memory utilization.

The plots created by the posterior predictive check during the fast diagnosis step in tutorial 5 give an impression of how well the posterior performs. Increasing `thin`, `init_strategy_num_candidates` and `warmup_steps` or changing the `init_strategy` might improve results. For a start, you might want to keep the default values.

### Access the posterior

The subsequent cell allows to access the posterior. `ddm_stride/sbi_extensions/mcmc.py` provides functions for sampling from the posterior or computing the potential function $\log P(x| \theta, \pi) \cdot P(\theta)$.

In [1]:
import hydra
import torch
from ddm_stride.pipeline.evaluate import load_experimental_data
from ddm_stride.pipeline.infer import build_posterior, build_prior
from ddm_stride.utils.data_names import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with hydra.initialize(config_path='../config'):
    cfg = hydra.compose(config_name='config')

# Load the experimental data
experimental_data = load_experimental_data(cfg)

# exp_cond contains the experimental conditions, if available
if len(get_experimental_condition_names(cfg)) > 0:
    exp_cond = torch.Tensor(experimental_data[:, get_experimental_condition_names(cfg)].values)
else: 
    exp_cond = None

# x contains the observations
x = torch.Tensor(experimental_data.loc[:, get_observation_names(cfg)].values)

# Build the posterior object. You can pass default observations and experimental conditions via 
# posterior = build_posterior(cfg, x=x, exp_cond=exp_cond).
# As shown below, x and exp_cond can also be set when sampling or computing probabilities.
posterior = build_posterior(cfg)

In [3]:
# Sample from the posterior
# If you want to draw many samples, you might want to increase the number of chains and workers.
samples = posterior.sample((10,), x, exp_cond, num_chains=1, num_workers=1)
print(f"samples: {samples}")

Running 1 MCMC chains in 1 batches.: 100%|██████████| 1/1 [04:12<00:00, 252.77s/it]

samples: tensor([[-0.8032,  1.7580,  0.6998],
        [-0.7950,  1.7599,  0.7000],
        [-0.8324,  1.7744,  0.6999],
        [-0.7936,  1.7630,  0.6998],
        [-0.8173,  1.7626,  0.6996],
        [-0.8201,  1.7639,  0.7000],
        [-0.7998,  1.7473,  0.7000],
        [-0.8237,  1.7621,  0.6999],
        [-0.8218,  1.7607,  0.6998],
        [-0.8209,  1.7577,  0.6997]])


In [4]:
# Sample a two parameters from the prior
theta = build_prior(cfg, device='cpu').sample((2,))
print(f"theta: {theta}")

# Compute the log probability for each parameter
log_prob = posterior.log_prob(theta, x, exp_cond)
potential = posterior.potential(theta, x, exp_cond)

print(f"log_prob: {log_prob}, \npotential: {potential}")

theta: tensor([[1.1995, 1.2180, 0.4649],
        [0.5047, 1.3627, 0.6425]])
log_prob: tensor([[-10481.7188,  -9959.1328]]), 
potential: tensor([[-10481.7188,  -9959.1328]])
